# Comparing fully specified and partially specified update functions

In this notebook, we demonstrate how to use symbolic representation in AEON to compare update functions.

In particular, we want to answer the question "can a partially specified funciton `f` be instantiated such that it is (semantically) equivalent to a fully specified function `g`". Or, even more generally, if `g` is also partially specified, which instantiations of `f` and `g` are (semantically) equivalent?

To work with update functions, we first need to create a Boolean network. However, the contents of this Boolean network are not relevant, we only use it to "declare" the variables and functions we are working with, and nothing else.

In [1]:
from biodivine_aeon import *

In [2]:
rg = RegulatoryGraph(["a", "b", "c", "d", "e"])
# The purpose of these regulations and update functions is explained later, but they are not relevant
# for what we will be testing, they are merely used to "convince" our symbolic representation that
# the network uses all declared variables and functions.
rg.add_regulation({ 'source': "b", 'target': "a" })
rg.add_regulation({ 'source': "c", 'target': "a" })
rg.add_regulation({ 'source': "d", 'target': "a" })
rg.add_regulation({ 'source': "a", 'target': "e" })
bn = BooleanNetwork(rg)
bn.add_explicit_parameter("p1", 3)
bn.add_explicit_parameter("p2", 1)
bn.set_update_function("a", "p1(b,c,d)")
bn.set_update_function("e", "p2(a)")
print(bn)

BooleanNetwork(variables=5, regulations=4, explicit_parameters=2, implicit_parameters=3)


Now we can parse the functions that we are interested in, and turn them into BDDs:

In [3]:
partial = UpdateFunction(bn, "a & p1(c,d,e)")
compatible = UpdateFunction(bn, "a & (c | d) & !e")
incompatible = UpdateFunction(bn, "!a & c")

print(partial)

a & p1(c, d, e)


Now we need to build the `SymbolicContext` such that we have a mapping from Boolean network variables and functions to the BDD variables.

In [4]:
# At this point, symbolic context could complain that we have declared function `f`, but we are not using it.
# In the future, we will be able to just force-create the context anyway. For now, we need to add some update
# function that will actually use `f`, which is what we did above.
ctx = SymbolicContext(bn)

# We can also get the list of "symbolic variables" that are used to represent the "network variables" as well as
# the instantiations of the individual unknown functions:
state_variables = ctx.network_bdd_variables()
id_p1 = bn.find_explicit_parameter("p1")
id_p2 = bn.find_explicit_parameter("p2")
p1_variables = [ var for (inputs, var) in  ctx.get_function_table(id_p1) ]
p2_variables = [ var for (inputs, var) in  ctx.get_function_table(id_p2) ]
print(len(state_variables), len(p1_variables), len(p2_variables))

5 8 2


Once we have that, we can actually transform the functions into BDDs:

In [5]:
partial_bdd = ctx.mk_update_function(partial)
compatible_bdd = ctx.mk_update_function(compatible)
incompatible_bdd = ctx.mk_update_function(incompatible)
print(partial_bdd)
print(compatible_bdd)
print(incompatible_bdd)

Bdd(vars = 18, len = 512, cardinality = 65536)
Bdd(vars = 18, len = 6, cardinality = 49152)
Bdd(vars = 18, len = 4, cardinality = 65536)


In this representation, checking whether partial function `f` can be instantiated as concrete function `g` is actually rather easy:

In [6]:
# 1. Compute f(x) <=> g(x). This gives us a BDD which is satisfied for every combination of 
# function inputs (state_variables) and instantiations (p1_variables) for which functions 
# `f` and `g` return the same value.
compatible_instantiations = partial_bdd.l_iff(compatible_bdd)
# 2. We eliminate the function inputs (state variables) from the BDD, such that we only keep those
# instantiations for which we have *all* input combinations present (i.e. the whole function is
# equivalent, not just some specific input-output pairs).
compatible_instantiations = compatible_instantiations.r_for_all(state_variables)

# If the set is empty, it is equivalent to false. This set should not be empty.
print(compatible_instantiations.is_false())

# However, this set should be empty because the two functions are incompatible.
incompatible_instantiations = partial_bdd.l_iff(incompatible_bdd)
incompatible_instantiations = incompatible_instantiations.r_for_all(state_variables)
print(incompatible_instantiations.is_false())

False
True


Finally, we can actually look at what the specific instantiations of `p1` that cause `f` to be equivalent with `g` look like.

Here, we have to again "hack" the result a little bit, because at the moment, we can only iterate over instantiated full update functions of network variables, not any arbitrary uninterpreted function. However, in our "dummy" network, variable `a` has an update function that is exactly `p1`. Hence we can ask for the update function of `a` and it will in fact give us valid instantiations of `p1`.

Typically, there should be only one instantiation in the resulting set, but in some rare cases, there could be multiple instances of `p1` which actually lead to the same update function (e.g. if `p1` appears multiple times within some more complicated expression, or if `p1` is not really relevant for the result of the function). Another instance where there could be multiple instantiations is if the second function is also partially specified, in which case we get the instantiations of `f` for which there exists *some* instantiation of `g` such that `f` and `g` are equivalent.

Also note that the resulting expression is not syntactically equivalent to our original function, but we can transform it in such a way that it is: `a & p1(c,d,e)` instantiates to `a & ((!c & d & !e) | (c & !e))`, which is equivalent to `a & !e & ((!c & d) | c)`, which is equivalent to `a & (c | d) & !e`.


In [7]:
stg = AsynchronousGraph(bn)
symbolic_set = ColorSet(stg.symbolic_context(), compatible_instantiations)
for function in symbolic_set.items(retained=["p1"]):
    print(function)    

ColorModel({'p1': '((!x_0 & (x_1 & !x_2)) | (x_0 & !x_2))'})
